<https://colab.research.google.com/github/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial3/Tutorial3.ipynb>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class GATLayer(nn.Module):
    """
    Simple PyTorch Implementation of the Graph Attention layer.
    """

    def __init__(self):
        super(GATLayer, self).__init__()

    def forward(self, input, adj):
        print("")

In [3]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features)))  #xavier paramiter inizializator
nn.init.xavier_uniform_(W.data, gain=1.414)

input = torch.rand(nb_nodes, in_features)

# linear transformation
h = torch.mm(input, W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [4]:
a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))  #xavier paramiter inizializator
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)

leakyrelu = nn.LeakyReLU(0.2)  # LeakyReLU

torch.Size([4, 1])


In [5]:
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)

In [6]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2))

In [7]:
print(a_input.shape, a.shape)
print("")
print(torch.matmul(a_input, a).shape)
print("")
print(torch.matmul(a_input, a).squeeze(2).shape)

torch.Size([3, 3, 4]) torch.Size([4, 1])

torch.Size([3, 3, 1])

torch.Size([3, 3])


In [8]:
# Masked Attention
adj = torch.randint(2, (3, 3))

zero_vec = -9e15 * torch.ones_like(e)
print(zero_vec.shape)

torch.Size([3, 3])


In [9]:
attention = torch.where(adj > 0, e, zero_vec)
print(adj, "\n", e, "\n", zero_vec)
attention

tensor([[1, 0, 0],
        [1, 1, 0],
        [1, 1, 0]]) 
 tensor([[8.3964, 8.8363, 5.5255],
        [8.6860, 9.1260, 5.8152],
        [6.0174, 6.4574, 3.1466]], grad_fn=<LeakyReluBackward0>) 
 tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]])


tensor([[ 8.3964e+00, -9.0000e+15, -9.0000e+15],
        [ 8.6860e+00,  9.1260e+00, -9.0000e+15],
        [ 6.0174e+00,  6.4574e+00, -9.0000e+15]], grad_fn=<SWhereBackward0>)

In [10]:
attention = F.softmax(attention, dim=1)
h_prime = torch.matmul(attention, h)

In [11]:
attention, h_prime

(tensor([[1.0000, 0.0000, 0.0000],
         [0.3917, 0.6083, 0.0000],
         [0.3917, 0.6083, 0.0000]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.3690, 3.3990],
         [0.5647, 3.4041],
         [0.5647, 3.4041]], grad_fn=<MmBackward0>))

In [12]:
print(h_prime,"\n",h)

tensor([[0.3690, 3.3990],
        [0.5647, 3.4041],
        [0.5647, 3.4041]], grad_fn=<MmBackward0>) 
 tensor([[0.3690, 3.3990],
        [0.6907, 3.4073],
        [0.0219, 1.3892]], grad_fn=<MmBackward0>)
